In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install PyPDF2 pdfplumber pytesseract opencv-python pdf2image

In [ ]:
import os
import pandas as pd
import re

In [ ]:
import pdfplumber
import re
from pdf2image import convert_from_path
import pytesseract

In [ ]:
def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n" if page.extract_text() else ""
    return text

In [ ]:
def extract_text_from_scanned_pdf(file_path):
    images = convert_from_path(file_path)
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image) + "\n"
    return text

In [ ]:
def extract_text(file_path):
    with open(file_path, "rb") as f:
        try:
            if pdfplumber.open(f).pages[0].extract_text():
                return extract_text_from_pdf(file_path)
            else:
                return extract_text_from_scanned_pdf(file_path)
        except:
            return extract_text_from_scanned_pdf(file_path)

In [ ]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [ ]:
def extract_invoice_data(text):
    invoice_data = {}

    invoice_data['invoice_number'] = re.search(r'Invoice #:\s*([A-Z0-9\-]+)', text)
    invoice_data['invoice_date'] = re.search(r'Invoice Date:\s*([\d\s\w]+)', text)
    invoice_data['due_date'] = re.search(r'Due Date:\s*([\d\s\w]+)', text)
    invoice_data['customer_name'] = re.search(r'Customer Details:\s*([\w\s]+)', text)
    invoice_data['taxable_amount'] = re.search(r'Taxable Amount ₹([\d,.]+)', text)
    invoice_data['cgst'] = re.search(r'CGST\s*\d+(\.\d+)?%\s*₹([\d,.]+)', text)
    invoice_data['sgst'] = re.search(r'SGST\s*\d+(\.\d+)?%\s*₹([\d,.]+)', text)
    invoice_data['total_amount'] = re.search(r'Total ₹([\d,.]+)', text)
    invoice_data['total_discount'] = re.search(r'Total Discount ₹([\d,.]+)', text)

    return {k: v.group(1) if v else None for k, v in invoice_data.items()}

In [ ]:
def process_invoice(file_path):
    extracted_text = extract_text(file_path)
    cleaned_text = clean_text(extracted_text)
    print(cleaned_text)
    invoice_data = extract_invoice_data(cleaned_text)
    return invoice_data

In [ ]:
folder_path = '/content/drive/MyDrive/Jan to Mar'
all_invoices_data = []

In [ ]:
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        file_path = os.path.join(folder_path, filename)
        data = process_invoice(file_path)
        all_invoices_data.append(data)

TAX INVOICE ORIGINAL FOR RECIPIENT UNCUE DERMACARE PRIVATE LIMITED GSTIN 23AADCU2395N1ZY C/o KARUNA GUPTA KURELE, 1st Floor S.P Bungalow Ke Pichhe, Shoagpur Shahdol, Shahdol Shahdol, MADHYA PRADESH, 484001 Mobile +91 8585960963 Email ruhi@dermaq.in Invoice #: INV-146 Invoice Date: 29 Mar 2024 Due Date: 29 Mar 2024 Customer Details: Abhikaran Jalonha Place of Supply: 23-MADHYA PRADESH # Item Rate / Item Qty Taxable Value Tax Amount Amount Cetaphil DAM Advance Ultra-Hydrati- 395.08 1 1 TUB 395.08 71.12 (18%) 466.20 ng Lotion Face - 100 gm 438.98 (-10%) Ahaglow Advanced Skin Rejuvenating 556.34 2 1 PAC 556.34 100.14 (18%) 656.48 Face Wash Gel 632.20 (-12%) 206.43 3 Acutret 20 mg - 10 capsules 6 STRP 1,238.57 148.63 (12%) 1,387.20 258.04 (-20%) 695.80 4 Faireye under eye cream 1 695.80 125.24 (18%) 821.04 790.68 (-12%) 462.37 5 Biluma cream - 15 gm 1 PAC 462.37 83.23 (18%) 545.60 525.42 (-12%) Taxable Amount ₹3,348.16 CGST 6.0% ₹74.31 SGST 6.0% ₹74.31 CGST 9.0% ₹189.86 SGST 9.0% ₹189.86 Ro

In [ ]:
invoices_df = pd.DataFrame(all_invoices_data)

In [ ]:
invoices_df.head()

,invoice_number,invoice_date,due_date,customer_name,taxable_amount,cgst,sgst,total_amount,total_discount
0,INV-146,29 Mar 2024 Due Date,29 Mar 2024 Customer Details,Abhikaran Jalonha Place of Supply,"3,348.16",.0,.0,"3,877.00",674.48
1,INV-145,28 Mar 2024 Due Date,28 Mar 2024 Customer Details,Indraja Mohite Place of Supply,"1,917.86",.0,.0,"2,141.00",520.20
2,INV-140,06 Mar 2024 Due Date,06 Mar 2024 Customer Details,Ankit Place of Supply,999.36,.0,.0,"1,148.00",286.99
3,INV-136,15 Feb 2024 Due Date,04 Mar 2024 Customer Details,Rishabh Ramola Ph,961.36,.0,.0,"1,134.00",283.60
4,INV-143,28 Mar 2024 Due Date,28 Mar 2024 Customer Details,Prashant Place of Supply,"6,563.98",.0,.0,"7,612.00",880.00


In [ ]:
invoices_df.to_csv('invoices_data.csv', index=False)